Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2019.1

Professor: André Lage Freitas

Aluno: Igor da Cunha Araújo Theotônio


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de carga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [1]:
# Espaço para escrever o código fonte.

#= 
    Installing and adding required packages for problem resolution
=#
using Pkg
Pkg.add("Distributed")
using Distributed

#= 
    Adds 3 workers (goes from 2 to 4 because 1 is the master)
=#
addprocs(3)

#= 
    Initializing workloads array with zeros, it's the size of number of workers
    and it will hold the workloads of every worker
    Also sets the first leader to be the worker with the greatest id, in this case
    worker 4
    And it also create another array with false values, with the size of number
    of workers, it will hold which worker is participating or not in the election
=#
@everywhere workloads = fill(0.0, nprocs() - 1)
@everywhere global leader = nprocs()
@everywhere candidates = fill(false, nprocs() - 1)

#= 
    Main loop function, it will generate randomly new workloads for each worker
    and communicates the change for the others.
=#
@everywhere function failure_detection()
    while true
        my_id = myid()
        workloads[my_id - 1] = round(rand(1)[1], digits = 1)
        
        for worker in workers()
            if worker != my_id
                @spawnat worker update_workloads(my_id, workloads[my_id - 1])
            end
        end
        
        sleep(2)

        message = string("Workload from Worker ", my_id, ": ", workloads)
        @spawnat 1 println(message)
    end
end

#= 
    This function not only updates the workloads, it checks the conditions
    to start an election, so if the leader workload is greater or equal than 
    0.8 and the executing process workload is lower than 0.8 it starts
=#
@everywhere function update_workloads(id, workload)
    workloads[id - 1] = workload
    if workloads[leader - 1] >= 0.8 && workloads[myid() - 1] < 0.8
        candidates[myid() - 1] = true
        begin_election()
    end
end

#= 
    This funtion sends process id to its neighbor so the election trully
    starts with the first candidate
=#
@everywhere function begin_election()
    current_id = myid()
    next_id = current_id + 1
    
    # Since its an circular election we need to check if we are in the last worker to send to the first one
    if current_id == nprocs()
        next_id = 2
    end
    
    @spawnat next_id election(current_id)
end

#= 
    The election itself, responsible for checking conditions, updating leader candidates
    and sending messages to worker neighbors
=#
@everywhere function election(previous_id)
    candidates[previous_id - 1] = true
    
    current_id = myid()
    next_id = myid() + 1
    if current_id == nprocs()
        next_id = 2
    end
    
    # If the worker called an election on itself, it's the new elected leader
    if previous_id == current_id
        global leader = current_id
        
        # It's needed to update leader value for every process
        for worker in workers()
            @spawnat worker update_leader(leader, current_id)
        end
        message = string("New leader elected! Worker ", current_id)
        @spawnat 1 println(message)
    else
        #= 
            If the executing worker workload is lower than the workload of the new leader
            probable best candidate, it changes the message to its id and mark itself as 
            candidate sending the message to the next worker
        =#
        if workloads[previous_id - 1] > workloads[current_id - 1]
            candidates[current_id - 1] = true
            @spawnat next_id election(current_id)
            
        #= 
            If the executing worker workload is greater than the workload of the new leader
            probable best candidate, it sends the message to the next worker and mark itself 
            as candidate
        =#
        elseif workloads[previous_id - 1] < workloads[current_id - 1]
            candidates[current_id - 1] = true
            @spawnat next_id election(previous_id)
        else
        #= 
            If the executing worker workload is equal to the workload of the new leader
            probable best candidate, it checks id conditions
        =#
            
            #= 
                If the executing worker id is lower than the id of the new leader probable
                best candidate, it sends the message to the next worker and marks itself
                as a candidate
            =#
            if previous_id > current_id
                candidates[current_id - 1] = true
                @spawnat next_id election(previous_id)
                
            #= 
                If the executing worker id is greater than the id of the new leader probable
                best candidate AND it isn't already a candidate, it changes the message to its 
                id, sends the message to the next worker and marks itself as a candidate
            =#
            elseif previous_id < current_id && !candidates[current_id - 1]
                candidates[current_id - 1] = true
                @spawnat next_id election(current_id)
                
            #= 
                If the executing worker id is greater than the id of the new leader probable
                best candidate AND it is already a candidate, it does nothing
            =#
            elseif previous_id < current_id && candidates[current_id - 1]
            end
        end
    end
end

# Updates the leader and marks processes as not candidates
@everywhere function update_leader(leader_id, previous_id)
    current_id = myid()
    next_id = myid() + 1
    global leader;
    
    if current_id == nprocs()
        next_id = 2
    end
    
    leader = leader_id
    candidate[previous_id - 1] = false
    candidate[current_id - 1] = false
end

# Starts the execution
for worker in workers()
    @spawnat worker failure_detection()
end

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]
Workload from Worker 2: [0.0, 0.8, 0.0]
Workload from Worker 4: [0.5, 0.0, 0.0]
Workload from Worker 3: [1.0, 0.8, 0.0]
Workload from Worker 2: [1.0, 0.9, 0.0]
Workload from Worker 4: [0.9, 0.9, 0.0]
Workload from Worker 3: [0.9, 0.9, 0.8]
Workload from Worker 2: [0.9, 0.9, 0.8]
Workload from Worker 4: [0.3, 0.9, 0.8]
Workload from Worker 3: [0.3, 0.9, 0.9]
New leader elected! Worker 2
Workload from Worker 2: [0.3, 0.2, 0.9]
Workload from Worker 4: [0.6, 0.2, 0.9]
Workload from Worker 3: [0.6, 0.2, 0.4]
Workload from Worker 2: [0.6, 0.7, 0.4]
Workload from Worker 4: [0.7, 0.7, 0.4]
Workload from Worker 3: [0.7, 0.7, 1.0]
Workload from Worker 2: [0.7, 0.8, 1.0]
Workload from Worker 4: [0.5, 0.8, 1.0]
Workload from Worker 3: [0.5, 0.8, 1.0]
Workload from Worker 2: [0.5, 0.8, 1.0]
Workload from Worker 4: [0.3, 0.8, 1.0]
Workload from Worker 3: [0.3, 0.8, 0.3]
W


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.